In [9]:
import os
import cv2 as cv
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from math import log10, sqrt 
import tensorflow as tf

def load_images_from_folder(foldername, target_size=(64, 64)):
    images = []
    for filename in os.listdir(foldername):
        img = cv.imread(os.path.join(foldername, filename), cv.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv.resize(img, target_size)  
            images.append(img)
    return images
# https://stackoverflow.com/questions/66471454/how-to-read-images-from-our-system-folder-to-jupyter-notebook-python-file

# PSNR function from previous submission, modified for Tensor use 
def PSNR(original, compressed):
    original = tf.cast(original, tf.float32)
    compressed = tf.cast(compressed, tf.float32)      
    mse = tf.reduce_mean(tf.square(original - compressed))
    max_pixel = 255.0
    psnr = 20 * tf.math.log(max_pixel / tf.sqrt(mse)) / tf.math.log(10.0)

    return psnr 

input_img = Input(shape=(64, 64, 1))  

def encode (input):
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    return encoded

def forward(input):
    encoded = encode(input)
   
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    return decoded
    # https://www.tensorflow.org/api_docs/python/tf

# Build autoencoder model
autoencoder = Model(input_img, forward(input_img))
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')


# Load images
images=load_images_from_folder("pneumonia")
images = np.array(images)
images = images.astype('float32') / 255.
images = np.expand_dims(images, axis=-1)

# Generate noisy images
lam25noise = np.random.poisson(25, images.shape)
lam25 = images + lam25noise

lam50noise = np.random.poisson(50, images.shape)
lam50 = images + lam50noise

lam75noise = np.random.poisson(75, images.shape)
lam75 = images + lam75noise
# https://stackoverflow.com/questions/19289470/adding-poisson-noise-to-an-image

# Split the dataset into training and testing sets (80% train, 20% test)
x_train, x_test = train_test_split(images, test_size=0.2, random_state=42)
train25, test25 = train_test_split(lam25, test_size=0.2, random_state=42)
train50, test50 = train_test_split(lam50, test_size=0.2, random_state=42)
train75, test75 = train_test_split(lam75, test_size=0.2, random_state=42)

# Original Images

In [15]:
model = Model(input_img, forward(input_img))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(x_train, x_train,
                epochs=5,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))


encoded_images = encode(x_train)
encoded_images = tf.reshape(encoded_images, [-1])

np.savetxt("encoded_images.csv", encoded_images, delimiter=",")

# Validation PSNR for lambda 25
decoded_images_25 = model.predict(test25) 
psnr_25 = PSNR(test25, decoded_images_25)
print("Average PSNR for lambda 25 validation images:", np.mean(psnr_25))

Epoch 1/5
15/15 [==============================] - 5s 246ms/step - loss: 0.0425 - val_loss: 0.0253
Epoch 2/5
15/15 [==============================] - 4s 236ms/step - loss: 0.0192 - val_loss: 0.0117
Epoch 3/5
15/15 [==============================] - 4s 245ms/step - loss: 0.0093 - val_loss: 0.0074
Epoch 4/5
15/15 [==============================] - 4s 278ms/step - loss: 0.0070 - val_loss: 0.0060
Epoch 5/5
15/15 [==============================] - 1s 28ms/step
Average PSNR for lambda 25 validation images: 20.164845


# Lambda = 25

In [ ]:
model25 = Model(input_img, forward(input_img))
model25.compile(optimizer='adam', loss='mean_squared_error')

model25.fit(train25, x_train,
                epochs=5,
                batch_size=128,
                shuffle=True,
                validation_data=(test25, x_test))

psnr25 = model25.evaluate(test25, x_test)
print("PSNR:", psnr25)

encoded_images_noisy = []
for image in train25:  
    encoded_image = encode(np.expand_dims(image, axis=0)) 
    encoded_image_flat = np.array(encoded_image).flatten() 
    encoded_images_noisy.append(encoded_image_flat)

encoded_images_noisy = np.array(encoded_images_noisy)

np.savetxt("encoded_images_lam25.csv", encoded_images_noisy, delimiter=",")

# Validation PSNR for lambda 25
decoded_images_25 = model25.predict(test25) 
psnr_25 = PSNR(test25, decoded_images_25)
print("Average PSNR for lambda 25 validation images:", np.mean(psnr_25))

Epoch 1/5
15/15 [==============================] - 5s 342ms/step - loss: 0.1598 - val_loss: 0.0463
Epoch 2/5
15/15 [==============================] - 5s 316ms/step - loss: 0.0389 - val_loss: 0.0361
Epoch 3/5
15/15 [==============================] - 5s 311ms/step - loss: 0.0339 - val_loss: 0.0319
Epoch 4/5
15/15 [==============================] - 5s 332ms/step - loss: 0.0322 - val_loss: 0.0313
Epoch 5/5
15/15 [==============================] - 0s 20ms/step - loss: 0.0312
PSNR: 0.031163414940238
15/15 [==============================] - 0s 22ms/step
Average PSNR for lambda 25 validation images: 19.99965


# Lambda = 50

In [ ]:
model50 = Model(input_img, forward(input_img))
model50.compile(optimizer='adam', loss='mean_squared_error')

model50.fit(train50, x_train,
                epochs=5,
                batch_size=128,
                shuffle=True,
                validation_data=(test50, x_test))

psnr50 = model50.evaluate(test50, x_test)
print("PSNR:", psnr50)

encoded_images_noisy = []
for image in train50:  
    encoded_image = encode(np.expand_dims(image, axis=0)) 
    encoded_image_flat = np.array(encoded_image).flatten() 
    encoded_images_noisy.append(encoded_image_flat)

encoded_images_noisy = np.array(encoded_images_noisy)

np.savetxt("encoded_images_lam50.csv", encoded_images_noisy, delimiter=",")

# Validation PSNR for lambda 50
decoded_images_50 = model50.predict(test50)  
psnr_50 = PSNR(test50, decoded_images_50)
print("Average PSNR for lambda 50 validation images:", np.mean(psnr_50))

Epoch 1/5
15/15 [==============================] - 5s 332ms/step - loss: 0.2788 - val_loss: 0.2783
Epoch 2/5
15/15 [==============================] - 5s 342ms/step - loss: 0.2807 - val_loss: 0.2774
Epoch 3/5
15/15 [==============================] - 5s 302ms/step - loss: 0.2802 - val_loss: 0.2772
Epoch 4/5
15/15 [==============================] - 5s 302ms/step - loss: 0.2801 - val_loss: 0.2772
Epoch 5/5
15/15 [==============================] - 0s 20ms/step - loss: 0.2771
PSNR: 0.27712565660476685
15/15 [==============================] - 0s 21ms/step
Average PSNR for lambda 50 validation images: 13.9863615


# Lambda = 75

In [ ]:
model75 = Model(input_img, forward(input_img))
model75.compile(optimizer='adam', loss='mean_squared_error')

model75.fit(train75, x_train,
                epochs=5,
                batch_size=128,
                shuffle=True,
                validation_data=(test75, x_test))

psnr75 = model75.evaluate(test75, x_test)
print("PSNR:", psnr75)

encoded_images_noisy = []
for image in train75:  
    encoded_image = encode(np.expand_dims(image, axis=0)) 
    encoded_image_flat = np.array(encoded_image).flatten() 
    encoded_images_noisy.append(encoded_image_flat)

encoded_images_noisy = np.array(encoded_images_noisy)
np.savetxt("encoded_images_lam75.csv", encoded_images_noisy, delimiter=",")

# Validation PSNR for lambda 75
decoded_images_75 = model50.predict(test75)  
psnr_75 = PSNR(test75, decoded_images_75)
print("Average PSNR for lambda 75 validation images:", np.mean(psnr_75))

Epoch 1/5
15/15 [==============================] - 5s 307ms/step - loss: 0.0660 - val_loss: 0.0345
Epoch 2/5
15/15 [==============================] - 4s 299ms/step - loss: 0.0340 - val_loss: 0.0319
Epoch 3/5
15/15 [==============================] - 5s 302ms/step - loss: 0.0322 - val_loss: 0.0312
Epoch 4/5
15/15 [==============================] - 4s 297ms/step - loss: 0.0317 - val_loss: 0.0310
Epoch 5/5
15/15 [==============================] - 0s 19ms/step - loss: 0.0309
PSNR: 0.030855482444167137
15/15 [==============================] - 0s 20ms/step
Average PSNR for lambda 75 validation images: 10.519686


##### For this particular task,  how did you measure how good your configuration for latent space dimensionality is? 

One determinant of how good the latent space configuration is the ammount of loss. Since the loss of the model seems to be minimal, therefore the configuration is appropriate